In [1]:
import json
from research.model_creator.models import Model

editor_json = "./cz/intro-cz.json"

with open(editor_json, "r", encoding="utf-8") as f:
    model = Model("xx", "stage", json.load(f))


ModuleNotFoundError: No module named 'research'

In [2]:
decision_nodes = model.nlp_model.dm_examples.items()
intent_nodes = model.nlp_model.intent_examples.items()

In [3]:
intent_nodes


dict_items([('stop', ['stop', ' quit', 'i want to quit', ' exit', 'i want to exit', ' close', 'i want to close', ' stop ', 'i want to stop ', ' stop it', 'i want to stop it', ' turn  off', 'i want to turn  off', ' turn you off', 'i want to turn you off', "don't talk", 'leave me be', "i've had enough of this", 'stop talking', "i don't want to play  anymore", "i don't want to play any game anymore", "i don't want to play  ", "i don't want to play any game ", "i don't like this game anymore", "i'm not enjoying this ", "i'm not enjoying this game", 'this is boring', 'bye bye', 'bye', 'goodbye', 'farewell']), ('Repeat', ['repeat', 'could you repeat that ', 'can you repeat that ', 'could you repeat it ', 'can you repeat it ', 'could you repeat the question ', 'can you repeat the question ', 'could you repeat that please', 'can you repeat that please', 'could you repeat it please', 'can you repeat it please', 'could you repeat the question please', 'can you repeat the question please', 'once 

In [6]:
for decision_node, possible_ways in decision_nodes:
    X = []
    y = []

    # load data from decision node + intent json
    for possible_way, text_examples in possible_ways.items():
        for text_example in text_examples:
            X.append(text_example.strip())
            y.append(possible_way + "_L")

    for intent, intent_examples in intent_nodes:
        print(intent)
        print("adding {}".format(len(intent_examples)))
        for text_example in intent_examples:
            X.append(text_example.strip())
            y.append(intent + "_G")

stop
adding 31
Repeat
adding 24
playOtherGame
adding 125
totalStop
adding 2
wait
adding 11
listOfGames
adding 66
gamesCounter
adding 12
volumeUp
adding 74
volumeDown
adding 124
timesPlayed
adding 21
stillThere
adding 9


In [7]:
len(X)

1319

In [8]:
len(y)

1319

In [14]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=8, shuffle=True, random_state=42)
split_indexes = kf.split(X, y)

for train_index, test_index in split_indexes:

    X_test, X_train = np.array(X)[np.array(test_index).astype(int)], np.array(X)[np.array(train_index).astype(int)]
    y_test, y_train = np.array(y)[np.array(test_index).astype(int)], np.array(y)[np.array(train_index).astype(int)]
    break



/home/petrlorenc/.virtualenvs/alquist/lib/python3.6/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=8.
  % (min_groups, self.n_splits)), UserWarning)


In [ ]:
dialogflow_format = {
  "name": "Yes-no-maybe-All-Corpus",
  "desc": "Petr Lorenc",
  "lang": "en",
  "sentences": []
}

for l, item in zip(y_train, X_train):
    dialogflow_format["sentences"].append({
      "text": item,
      "intent": l,
      "entities": [],
      "training": True
    })

for l, item in zip(y_test, X_test):
    dialogflow_format["sentences"].append({
      "text": item,
      "intent": l,
      "entities": [],
      "training": False
    })

with open("yes-no-maybe-limit-All-corpus.json", "w") as f:
    json.dump(dialogflow_format, f, sort_keys=False, indent=4, separators=(',', ': '), ensure_ascii=False)


In [31]:
limit = 20
cnt_label = {}


dialogflow_format = {
  "name": "Yes-no-maybe-{}-Corpus".format(limit),
  "desc": "Petr Lorenc",
  "lang": "en",
  "sentences": []
}

mapping_l = {"-3" : "YesNode", "-17": "NoNode", "-18": "MaybeNode"}


for l, item in zip(y_train, X_train):
    cnt_label[l] = cnt_label.get(l, 0) + 1
    if cnt_label[l] > limit:
        continue
    dialogflow_format["sentences"].append({
      "text": item,
      "intent": mapping_l[l] if l in mapping_l else l,
      "entities": [],
      "training": True
    })

for l, item in zip(y_test, X_test):
    if cnt_label[l] > limit:
        continue
    dialogflow_format["sentences"].append({
      "text": item,
      "intent": l,
      "entities": [],
      "training": False
    })

with open("yes-no-maybe-limit-{}-corpus.json".format(limit), "w") as f:
    json.dump(dialogflow_format, f, sort_keys=False, indent=4, separators=(',', ': '), ensure_ascii=False)

In [ ]:
limit = 20
cnt_label = {}


dialogflow_format = {
  "name": "Yes-no-maybe-{}-Corpus".format(limit),
  "desc": "Petr Lorenc",
  "lang": "en",
  "sentences": []
}

mapping_l = {"-3" : "YesNode", "-17": "NoNode", "-18": "MaybeNode"}


for l, item in zip(y_train, X_train):
    cnt_label[l] = cnt_label.get(l, 0) + 1
    if cnt_label[l] > limit:
        continue
    dialogflow_format["sentences"].append({
      "text": item,
      "intent": mapping_l[l] if l in mapping_l else l,
      "entities": [],
      "training": True
    })

for l, item in zip(y_test, X_test):
    if cnt_label[l] > limit:
        continue
    dialogflow_format["sentences"].append({
      "text": item,
      "intent": l,
      "entities": [],
      "training": False
    })

with open(dialogflow_format["name"] + ".json".format(limit), "w") as f:
    json.dump(dialogflow_format, f, sort_keys=False, indent=4, separators=(',', ': '), ensure_ascii=False)


In [29]:
heading = [" i think ",
           " i wonder ",
           " you know ",
           " sounds great ",
           " okay ",
           " so ",
           " right ",
           " believe me ",
           " at the end of the day ",
           " basically ",
           " well ",
           " would you let me know ",
           " i would like to hear ",
           " i want you to ",
           " could you ",
           " can you ",
           " in my humble opinion ",
           " for what it is worth ",
           " needless to say "]

footer = [" please ",
          " right ",
          " you know what I mean ",
          " you see ",
          " seriously ",
          " as you can see ",
          " you know ",
          " well "
          " right "]

dummy = [" yeah ", " oh ", " hmm ", " uhm ", " just ", " really ", " somehow "]


In [48]:
import random

limit = 15
cnt_label = {}


dialogflow_format = {
  "name": "Yes-no-maybe-{}-H-D-F-Corpus".format(limit),
  "desc": "Petr Lorencc",
  "lang": "en",
  "sentences": []
}

mapping_l = {"-3" : "YesNode", "-17": "NoNode", "-18": "MaybeNode"}


for l, item in zip(y_train, X_train):
    cnt_label[l] = cnt_label.get(l, 0) + 1
    if cnt_label[l] > limit:
        continue
    text = item
    text = (random.choice(heading) if random.random() > 0.5 else "") + text
    text = text + (random.choice(footer) if random.random() > 0.5 else "")

    dialogflow_format["sentences"].append({
      "text": text,
      "intent": mapping_l[l] if l in mapping_l else l,
      "entities": [],
      "training": True
    })

for l, item in zip(y_test, X_test):
    if cnt_label[l] > limit:
        continue
    text = item
    text = (random.choice(heading) if random.random() > 0.5 else "") + text
    text = text + (random.choice(footer) if random.random() > 0.5 else "")

    dialogflow_format["sentences"].append({
      "text": text,
      "intent": l,
      "entities": [],
      "training": False
    })

with open(dialogflow_format["name"] + ".json".format(limit), "w") as f:
    json.dump(dialogflow_format, f, sort_keys=False, indent=4, separators=(',', ': '), ensure_ascii=False)



In [52]:

import random

limit = 10
cnt_label = {}


dialogflow_format = {
  "name": "Yes-no-maybe-{}-H-D-F-Corpus".format(limit),
  "desc": "Petr Lorencc",
  "lang": "en",
  "sentences": []
}

mapping_l = {"-3" : "YesNode", "-17": "NoNode", "-18": "MaybeNode"}


for l, item in zip(y_train, X_train):
    cnt_label[l] = cnt_label.get(l, 0) + 1
    if cnt_label[l] > limit:
        continue
    text = item.split(" ")
    text.insert(random.randint(0, len(text)), random.choice(dummy) if random.random() > 0.5 else "")
    text = " ".join(text)
    text = (random.choice(heading) if random.random() > 0.5 else "") + text
    text = text + (random.choice(footer) if random.random() > 0.5 else "")

    dialogflow_format["sentences"].append({
      "text": text,
      "intent": mapping_l[l] if l in mapping_l else l,
      "entities": [],
      "training": True
    })

for l, item in zip(y_test, X_test):
    if cnt_label[l] > limit:
        continue
    text = item.split(" ")
    text.insert(random.randint(0, len(text)), random.choice(dummy) if random.random() > 0.5 else "")
    text = " ".join(text)
    text = (random.choice(heading) if random.random() > 0.5 else "") + text
    text = text + (random.choice(heading) if random.random() > 0.5 else "")

    dialogflow_format["sentences"].append({
      "text": text,
      "intent": l,
      "entities": [],
      "training": False
    })

with open(dialogflow_format["name"] + ".json".format(limit), "w") as f:
    json.dump(dialogflow_format, f, sort_keys=False, indent=4, separators=(',', ': '), ensure_ascii=False)



In [6]:
import json
with open("./from_editor/Yes-no-maybe-20-Corpus.json") as f:
    data = json.load(f)

In [7]:
train_X, train_y, test_X, test_y = [], [], [], []

for example in data["sentences"]:
    if example["training"]:
        train_X.append(example["text"])
        train_y.append(example["intent"])
    else:
        test_X.append(example["text"])
        test_y.append(example["intent"])

In [8]:
from collections import Counter

train_cnt = Counter(train_y)
test_cnt = Counter(test_y)

In [9]:
train_cnt


Counter({'yesNode': 20,
         'noNode': 20,
         'maybeNode': 11,
         'stop': 8,
         'Repeat': 6,
         'playOtherGame': 20,
         'totalStop': 1,
         'wait': 3,
         'listOfGames': 16,
         'gamesCounter': 3,
         'volumeUp': 19,
         'volumeDown': 20,
         'timesPlayed': 5,
         'stillThere': 2})

In [10]:
test_cnt

Counter({'maybeNode': 33,
         'stop': 23,
         'Repeat': 18,
         'totalStop': 1,
         'wait': 8,
         'listOfGames': 50,
         'gamesCounter': 9,
         'volumeUp': 55,
         'timesPlayed': 16,
         'stillThere': 7})

In [7]:

import json
with open("./from_editor/num_examples/paper-data-limit.json") as f:
    data = json.load(f)

In [8]:
train_X, train_y, test_X, test_y = [], [], [], []

for example in data["sentences"]:
    if example["training"]:
        train_X.append(example["text"])
        train_y.append(example["intent"])
    else:
        test_X.append(example["text"])
        test_y.append(example["intent"])

In [9]:
from collections import Counter

train_cnt = Counter(train_y)
test_cnt = Counter(test_y)

In [10]:
train_cnt


Counter({'-3_L': 281,
         '-17_L': 107,
         '-18_L': 22,
         'stop_G': 15,
         'Repeat_G': 12,
         'playOtherGame_G': 63,
         'totalStop_G': 1,
         'wait_G': 5,
         'listOfGames_G': 33,
         'gamesCounter_G': 6,
         'volumeUp_G': 37,
         'volumeDown_G': 62,
         'timesPlayed_G': 11,
         'stillThere_G': 4})

In [11]:
test_cnt

Counter({'-3_L': 282,
         '-17_L': 106,
         '-18_L': 22,
         'stop_G': 16,
         'Repeat_G': 12,
         'playOtherGame_G': 62,
         'totalStop_G': 1,
         'wait_G': 6,
         'listOfGames_G': 33,
         'gamesCounter_G': 6,
         'volumeUp_G': 37,
         'volumeDown_G': 62,
         'timesPlayed_G': 10,
         'stillThere_G': 5})

In [ ]:
3
